# Preparation

In [ ]:
import math
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
sys.path.insert(0,'../..') # The simulator package and package needed for analysis 
                           # (which are imported below) are in a parent folder

# Modified package from https://github.com/BilHim/trafficSimulator
from trafficSimulator import *
# Functions needed for simulation transition calculation
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [ ]:
# Specify the labels of the road segments in each route
all_routes = [[0, 3], [0, 1], [2, 3], [2, 1]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

# Levels of speed limit for fast and slow roads
# expressed as a proportion of the v_max set above
fast_track_factor, slow_track_factor = 1, 0.33 

# Length of the stop zone
stop_distance = 25 

# Waiting time for fast and slow roads (here set as the same)
fast_wait_time = 4
slow_wait_time = 4

# Total number of vehicles
vehicle_limit = 80

# Store vehicle specs to be input in the simulation package
vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

# Empty dataframe to store the simulation data
df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

# Initialize the probabilities for choosing each route for the 1st round;
# each route has an equal chance of being selected
vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.25, 0.25, 0.25, 0.25]]*vehicle_limit))

### Topology setup

In [ ]:
# Coordinates of the four corners of the traffic network; all four corners
# lie on either the x or the y axis
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

# Building the road lines with corresponding parameters
left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

## For the connections of the topology, we can represent with concrete lines, but
# the vehicles will not actually go through these routes; 
# they are  for demonstration purposes
connection_top_bottom = ((-1.25, bottom+2), (-1.25, top-2))
connection_bottom_top = ((1.25, top-2), (1.25, bottom+2))

### Utility params setup

In [ ]:
# Exploration factor in UCB alogorithm
delta = 0.25

# Penalty factor
alpha = 0

# Utility interpolation factor between money and time
gamma_mean, gamma_var = 5, 0.5

# Utility exponent
eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [ ]:
 def run_simulation(round_number, 
                    all_routes, 
                    vehicle_limit, 
                    df, 
                    vehicle_preferences,
                    vehicle_rate, 
                    vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global connection_top_bottom
    global connection_bottom_top
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        connection_top_bottom, # Road #4
        connection_bottom_top, # Road #5

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

In [ ]:
def simulation_with_changing_rate(vehicle_rates, 
                                  rounds,
                                  all_routes, 
                                  vehicle_limit, 
                                  df, 
                                  vehicle_preferences,
                                  vehicle_specs):
    """
    This function performs simulations for different given vehicle rates for a specified
    number of rounds and specified number of vehicles. After each round, the relevant data
    is saved.
    
    Parameters:
        vehicle_rates (list or array): a list of vehicle rates to be investigated
        rounds (integer): number of rounds to be run for each vehicle rate
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    for r in range(len(vehicle_rates)):
        rate = vehicle_rates[r]
        
        # Run the 1st round
        record_df = run_simulation(1, 
                                   all_routes, 
                                   vehicle_limit, 
                                   df, 
                                   vehicle_preferences,
                                   rate, 
                                   vehicle_specs)

        # Add meta data of the 1st round
        record_df['Round_number'] = 1
        record_df['vehicle_rate'] = rate

        # To better compare the performance of the simulations for all vehicle rates, the values
        # for utility exponent eta and utility interpolation factor gamma are only sampled once
        # since the distributions of these two parameters have not changed, after the values are
        # sampled, they are shared with the simulations of other vehicle rates
        if r == 0:
            # Initialize the needed columns
            utility_df = pd.DataFrame(columns=['Vehicle_label', 'Utilities', 'Probabilities'])

            utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
            utility_df['Routes_taken'] = [[]] * len(utility_df)
            utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
            utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)
            
            # Sample values for eta and gamma and make sure they are positive
            etas = []
            gammas = []

            for i in range(len(utility_df)):
                eta = np.random.normal(eta_mean, eta_var, 1)[0]
                while eta < 0:
                    eta = np.random.normal(eta_mean, eta_var, 1)[0]
                etas.append(eta)

            for i in range(len(utility_df)):
                gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
                while gamma < 0:
                    gamma = np.random.normal(gamma_mean, gamma_var, 1)[0]
                gammas.append(gamma)

            utility_df['Gamma'] = gammas
            utility_df['Eta'] = etas
        else:
            # Since the parameters values have already been sampled before, they will be reused for other vehicle rates
            # by reading into the saved data files
            utility_df = pd.read_csv(f"../1.1 Zero alpha/Saved_data/Utility_data/{vehicle_rates[0]}/Round_1.csv")

            utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
            utility_df['Routes_taken'] = [[]] * len(utility_df)
            utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
            utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)  

        # Summarize the collected data from the simulation and update the utility dataframe with the computed
        # utility values and probabilities
        utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            
        utility_tmp = utility_summary(all_routes, record_df)
        utility_df = update_utility_df(all_routes,
                                       delta, 
                                       alpha, 
                                       record_df=record_df,
                                       df=utility_df, 
                                       utility_tmp=utility_tmp)

        # Save the computed utility data
        if not os.path.exists(f'Saved_data/Utility_data/{rate}'):
            os.makedirs(f'Saved_data/Utility_data/{rate}')
        utility_df.to_csv(f"Saved_data/Utility_data/{rate}/Round_1.csv", index=False)

        # Updated probabilities for the 2nd round:
        vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

        # Finish the remaining rounds by repeating the process above
        for s in range(2, rounds+1):
            record_tmp = run_simulation(s, 
                                        all_routes, 
                                        vehicle_limit, 
                                        df, 
                                        vehicle_preferences,
                                        rate, 
                                        vehicle_specs)
            record_tmp['Round_number'] = s
            record_tmp['vehicle_rate'] = rate

            record_df = pd.concat([record_df, record_tmp], ignore_index=True)

            utility_tmp = utility_summary(all_routes, record_df)
            utility_df = update_utility_df(all_routes,
                                           delta, 
                                           alpha, 
                                           record_df=record_df,
                                           df=utility_df, 
                                           utility_tmp=utility_tmp,
                                           round_number=s)

            vehicle_preferences = dict(zip(range(vehicle_limit), 
                                           utility_df.Probabilities.to_list()))
            
            utility_df.to_csv(f"Saved_data/Utility_data/{rate}/Round_{s}.csv", index=False)

        # Export all the simulation data after finishing simulations for each vehicle rate
        record_df.to_csv(f"Saved_data/Simulation_records/VehicleRate_{rate}_Round_{s}.csv", index=False)

# Simulation runs

In [ ]:
simulation_with_changing_rate(vehicle_rates=np.append(np.arange(2,66,2), [1000]), 
                              rounds=50,
                              all_routes=all_routes, 
                              vehicle_limit=vehicle_limit, 
                              df=df, 
                              vehicle_preferences=vehicle_preferences,
                              vehicle_specs=vehicle_specs)